This is a basic tutorial for getting you started with **deviceparser.py**. Check out the README file or [this](https://colab.research.google.com/drive/1sbj9w4JpYKh5oqmP9doWr4BkHeUdcs8B) tutorial for more information.

In [0]:
# assuming that you have deviceparser.py in your working directory, load it as follows
from deviceparser import Device, ParsingFunctions

# import the other required libraries
import json
import re 
import numpy as np
import pandas as pd

In [0]:
# define the parsing functions
def parse_banner_batsize_hl(value):
  '''A function for parsing the batsize_hl

     Takes in str '3800' and returns the np.floats64 3800
  '''
  return {'batsize':np.float64(value)}


def parse_banner_displayres_hl(value):
  '''A function for parsing the displayres_hl

     Takes in str '1280x1920 pixels' and returns two np.float64's for the length and height of the screen
  '''
  res_pattern = re.compile('(\d+)x(\d+)')
  return_val =  re.findall(res_pattern, value)
  feature_names = ['displayres_len', 'displayres_height']
  if not return_val:
    return {'displayres_len':np.NaN, 'displayres_height':np.NaN}
  else:
    return  {name:np.float64(i) for name, i  in zip(feature_names, return_val[0])}
  

def parse_banner_ramsize_hl(banner_spec_value):
    '''A function for parsing the ramsize_hl

       Takes in str '2' and returns the np.float64 2
    '''
    return {'ramsize':np.float64(banner_spec_value)}
  
  
def parse_banner_displaysize_hl(banner_spec_value):
    '''A function for parsing the displaysize_hl
    
       Takes in str '6.0"' and returns np.float64 6.0
    '''
    disp_size_pattern = re.compile('(^\d+\.?\d*)\s?"')
    return_val = re.findall(disp_size_pattern, banner_spec_value)
    if not return_val:
      return {'displaysize':np.NaN}
    else:
      return {'displaysize':np.float64(return_val[0])}

In [0]:
# provide ParsingFunctions the user defined parsers

# current version of ParsingFunctions does not come preloaded with any methods 
print(ParsingFunctions.__dict__)

# ParsingFunctions can be provided a single parsing functions or a list of parsing functions
ParsingFunctions.add_new_parsers(parse_banner_batsize_hl)
print(ParsingFunctions.__dict__)

ParsingFunctions.add_new_parsers([parse_banner_displayres_hl, parse_banner_ramsize_hl, parse_banner_displaysize_hl])
print(ParsingFunctions.__dict__)

# only one parsing function of a given name can exist at a time inide ParsingFunctions
ParsingFunctions.add_new_parsers(parse_banner_batsize_hl)
# you have to reimport the ParsingFunctions class to redefine an existing function

In [0]:
# loading devices_data
file_path = '../devices_data.txt'
devices_dict = Device.read_devices_json(file_path)
devices_dict.keys()

In [0]:
# access maker_id and maker_name using Device.list_makers() method
makers = Device.list_makers(devices_dict)
print(makers)

In [0]:
# parsing a single device object

maker_name = 'Samsung'
# get Samsung's maker_id form the list we just created
for maker in makers:
  if maker[1] == maker_name:
    maker_id = maker[0]
    
# get the data for Galaxy S10 from the devices_dict 
device_name = 'Galaxy S10'
for device_num, device in devices_dict[maker_name].items():
  if device['device_name'] == device_name:
    device_id = maker_name.upper() + '_' + device_num 
    break
    
# create the device object by initializng an instance of the Device class
s10 = Device(device, device_id, maker_name, maker_id)

# new features have been created using the parsing functions
print(s10.batsize)
print(s10.displayres_height)
print(s10.displayres_len)
print(s10.displaysize)
print(s10.ramsize)

In [0]:
# creating a list of device objects for all makers
devices_collector = Device.create_devices_from_data(devices_dict)
print(len(devices_collector))
print(type(devices_collector[0]))

In [0]:
# simple one line filters can be written for devices collector using list comprehensions
# use getattr() for getting the value of a feature instead of self.feature_name
# this ensures that an AttributeError is not thrown if the device does not have the attribute 
samsung_devices = [x for x in devices_collector if getattr(x, 'maker_name', None) == 'Samsung']
print(len(samsung_devices) == len(devices_dict['Samsung'].keys()))

s10 = [x for x in devices_collector if getattr(x, 'device_name', None) == 'Galaxy S10']
print(s10[0].device_name)

batsize = [x.batsize if getattr(x, 'batsize', None) is not None else np.NaN for x in devices_collector]
print(batsize)

# this might seem complicated if you are beginner, but it is a great opportunity to learn about list comprehensions

In [0]:
# create a DataFrame after applying all the user defined parsing functions to the devices data
df = Device.create_df(devices_dict)
df.head()

In [0]:
df.info()